In [7]:
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
summarizer = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


(…)ilbart-cnn-12-6/resolve/main/config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

c:\Users\admin\anaconda3\lib\site-packages\huggingface_hub\file_download.py:138: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\admin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

(…)-12-6/resolve/main/tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

(…)tilbart-cnn-12-6/resolve/main/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

(…)tilbart-cnn-12-6/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [3]:
text = '''Text mining, also referred to as text data mining (abbr.: TDM), similar to text analytics, 
        is the process of deriving high-quality information from text. It involves 
        "the discovery by computer of new, previously unknown information, 
        by automatically extracting information from different written resources." 
        Written resources may include websites, books, emails, reviews, and articles. 
        High-quality information is typically obtained by devising patterns and trends 
        by means such as statistical pattern learning. According to Hotho et al. (2005)
        we can distinguish between three different perspectives of text mining: 
        information extraction, data mining, and a KDD (Knowledge Discovery in Databases) process.''' 

In [4]:
result = summarizer(text)
result

[{'summary_text': ' Text mining involves deriving high-quality information from text . Written resources may include websites, books, emails, reviews, and articles . Text mining is similar to text analytics . It involves the discovery by computer of new, previously unknown information by automatically extracting information from different written resources .'}]

In [5]:
len(text), len(result[0]['summary_text'])

(778, 341)

In [8]:
tokenizer = AutoTokenizer.from_pretrained("t5-small", model_max_length=512)
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

(…)small/resolve/main/tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

(…)ce.co/t5-small/resolve/main/spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

(…).co/t5-small/resolve/main/tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

(…)ace.co/t5-small/resolve/main/config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [9]:
type(tokenizer), type(model)

(transformers.models.t5.tokenization_t5_fast.T5TokenizerFast,
 transformers.models.t5.modeling_t5.T5ForConditionalGeneration)

In [10]:
preprocess_text = text.strip().replace("\n","")
input_text = "summarize: " + preprocess_text

In [11]:
tokenized_text = tokenizer.encode(input_text, return_tensors="pt").to(device)